# Домашнее задание (10 баллов)

1. (2 балла) Закончить реализацию `ClassificationDecisionTree` в decision_tree (реализовать feature_importance_, проверить корректность predict) и `RandomForestClassifier` в random_forest (predict/predict_proba). Обратите внимение, что в random_forest в качестве `base_estimator` предполагается использовать `DecisionTreeClassifier` из sklearn, использовать вашу реализацию решающего дерева необязательно. <br> Запуск тестов 
- `python -m unittest discover sem_dt_rf/decision_tree/tests`
- `python -m unittest discover sem_dt_rf/random_forest/tests`
 

In [ ]:
!python -m unittest discover sem_dt_rf/decision_tree/tests
!python -m unittest discover sem_dt_rf/random_forest/tests

- - -

2. (1 балл) Для регрессионного дерева необходимо использовать такой критерий:
    $$H(R) = \min_c \frac{1}{|R|} \sum_{(x_i, y_i) \in R} (y_i - c)^2$$
    
    Докажите, что минимум H(R) достигается при $c$:

    $$ c = \frac{1}{|R|} \sum_{(x_j, y_j) \in R} y_j$$

In [ ]:
# док-во: правая часть H(R) - выпуклая ф-я по c. ноль производной (по c) этой ф-ии - как раз среднее.

- - -

3. (3 балла) Реализуйте регрессионное дерево. В качестве критерия необходимо использовать критерий, определённый в пункте 2. В качестве функции выдачи результатов необходимо использовать среднее значение ответов по всем объектам в листе.

    Сгенерируйте однопризнаковую выборку для тестирования дерева и покажите работу дерева на этой выборке (пример см. ниже, можно использовать свою версию). Отобразите на одном графике значения алгоритма и точки. Что меняется при изменении параметра глубины? Сделайте выводы.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from decision_tree.decision_tree import RegressionDecisionTree
from ipywidgets import interact
%matplotlib inline

x_shape = 300
depths_from = 1
depths_to = 25

# generate example
x = np.arange(x_shape) / 100
y = x**3 * np.sin(x**3) + np.random.random(x_shape)

# store predictions for all depths
predicts_depth = {}
for n in range(depths_from, depths_to+1):
  predicts_depth[n] = RegressionDecisionTree(max_depth=n, min_leaf_size=4).fit(x, y).predict(x)

def update_plot(n=10):
  fig, ax = plt.subplots(figsize=(16,8))
  plt.scatter(x, y, color='g', label='true_values', s=10)
  plt.scatter(x, predicts_depth[n], label='pred_values', alpha=0.8, s=20)
  plt.title(f"max_depth = {n}")
  plt.legend()
  plt.xlabel("X")
  plt.ylabel("Y")
  plt.show()

# plot slider
interact(update_plot, n=(1,25,1));

interactive(children=(IntSlider(value=10, description='n', max=25, min=1), Output()), _dom_classes=('widget-in…

- при маленькой глуине сплиты делаются на участках с самой большой дисперсией (максимизируют MSE..)
- при увеличении глубины сплиты захватывают области с меньшей остаточной дисперсией (справа налево), bias модели, при этом, растет (тк в правой части имеем случайные выбросы, а лучше, наверное иметь просто прямую)
- при максимальной глубине области с меньшей дисперсией хорошо выучились деревом, в то время как области с большой дисперсией имеют меньше сплитов и являются источником бед модели (там в принципе меньше точек-данных)

вывод?
- если хочется зафитится больше к данным -> повышаем глубину
- если хотим иметь лучшую обобщающую способность -> уменьшаем глубину

- - -

4. (4 балла) Протестируйте различные реализации `random_forest` на `fetch_covtype` датасете (можно загрузить с помощью `sklearn.datasets.fetch_covtype`). Возможно, поможет ноутбук с семинара `ensembles_seminar.ipynb`. Для честного сравнения старайтесь использовать похожий набор гиперпараметров.
- ваша реализация (import `RandomForestClassifier as MyRandomForestClassifier` ниже)
- sklearn https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
- lightgbm https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMModel.html см. параметр `boosting_type`
- xgboost https://xgboost.readthedocs.io/en/stable/tutorials/rf.html

    Что нужно сделать: 
- Разбейте данные на train и test. 
- Оцените качество алгоритмов по метрике (balanced_accuracy_score)[https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html]
- Оцените время работы `train` и `predict`
- Сделайте выводы

In [1]:
# # import sys
# sys.path.extend(['/Users/alkrasnov/Documents/AM_ML_2_24']) # change your path

In [3]:
from sem_dt_rf.random_forest.random_forest import RandomForestClassifier as MyRandomForestClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import xgboost  as xgb

from sklearn.datasets import fetch_covtype
import pandas as pd
import numpy  as np

cov_type = fetch_covtype(data_home='_temp', download_if_missing=False)

from sklearn.preprocessing import LabelEncoder
target = LabelEncoder().fit_transform(cov_type.target)

X_train, X_test, y_train, y_test = train_test_split(cov_type.data, target, test_size=0.3, random_state=42)

In [4]:
import time
my_rf = MyRandomForestClassifier(n_estimators=100, max_depth=8, min_samples_leaf=50)

train_time = time.time()
my_rf.fit(X_train, y_train)
train_time = time.time()-train_time
print(f'my model, training time: {train_time:.2f} s')

predict_time = time.time()
my_preds = my_rf.predict(X_test)
predict_time = time.time()-predict_time
print(f'my model, predict time: {predict_time:.2f} s')

my_score = balanced_accuracy_score(y_test, my_preds)
print(f'my model, score: {my_score:.2f}')

my model, training time: 121.57 s
my model, predict time: 6.58 s
my model, score: 0.50


In [78]:
rf = RandomForestClassifier(n_estimators=100, max_depth=8, min_samples_leaf=50)

train_time = time.time()
rf.fit(X_train, y_train)
train_time = time.time()-train_time
print(f'sklearn, training time: {train_time:.2f} s')

predict_time = time.time()
rf_preds = rf.predict(X_test)
predict_time = time.time()-predict_time
print(f'sklearn, predict time: {predict_time:.2f} s')

rf_score = balanced_accuracy_score(y_test, rf_preds)
print(f'sklearn, score: {rf_score:.2f}')

sklearn, training time: 22.38 s
sklearn, predict time: 0.66 s
sklearn, score: 0.40


In [74]:
lgbt_params = {
    'num_leaves': 2**8,
    'min_data_in_leaf': 50,
    'objective': 'multiclass',
    'num_classes': len(np.unique(target)),
    'max_depth': 8,
    "boosting": "rf",
    "seed": 42,
    "bagging_frequency": 0.65,
    "subsample": .632,
    "subsample_freq": 1,
    "verbose": -1,
    "num_threads": -1
}

train_time = time.time()
lgbm = lgb.train(lgbt_params, lgb.Dataset(data=X_train, label=y_train), num_boost_round=100)
train_time = time.time()-train_time
print(f'lgbm, training time: {train_time:.2f} s')

predict_time = time.time()
lgbm_preds = lgbm.predict(X_test).argmax(axis=1)
predict_time = time.time()-predict_time
print(f'lgbm, predict time: {predict_time:.2f} s')

lgbm_score = balanced_accuracy_score(y_test, lgbm_preds)
print(f'lgbm, score: {lgbm_score:.2f}')

lgbm, training time: 14.53 s
lgbm, predict time: 1.21 s
lgbm, score: 0.80


In [73]:
xgb_params = {
    "booster": "gbtree",
    "subsample": .632,
    "max_depth": 8,
    "num_parallel_tree": 100,
    "objective": "multi:softmax",
    "num_class": len(np.unique(target)),
    "eta":1,
    "random_state": 42,
    "n_jobs": -1,
}

train_time = time.time()
xgbm = xgb.train(xgb_params, xgb.DMatrix(X_train, label=y_train), num_boost_round=1)
train_time = time.time()-train_time
print(f'lgbm, training time: {train_time:.2f} s')

predict_time = time.time()
xgbm_preds = xgbm.predict(xgb.DMatrix(X_test))
predict_time = time.time()-predict_time
print(f'lgbm, predict time: {predict_time:.2f} s')

xgbm_score = balanced_accuracy_score(y_test, xgbm_preds)
print(f'lgbm, score: {xgbm_score:.2f}')

lgbm, training time: 10.38 s
lgbm, predict time: 0.42 s
lgbm, score: 0.66


- sklearn: худший скор!, при этом время работы дольше чем у xgb. Нужно тщательнее настраивать гиперпараметры (и не использовать эту модель)
- наша модель: (работает на питоне) работает дольше всех, на дефолтных параметрах довольно скудный результат
- lgb: лучший скор, не лучшая скорость трейн и инференс, но сравнимая с xgb
- xgb: лучшая скорость трейн и инференс, но скор несравнимо ниже чем у lgb

выводы..
- лучше использовать lightgbm (либо CatBoost, это пока загадка)
- на совсем огромных датасетах, возможно, еще можно рассмотреть xgb (но, скорее всего, в lgb можно подобрать параметры для нужной скорости с сохранением скора)